In [1]:
# this is just to know how much time will it take to run this entire ipython notebook 
from datetime import datetime
# globalstart = datetime.now()
import pandas as pd
import numpy as np
import matplotlib

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

__Reading The Sampled data__

In [5]:
ratings = pd.read_csv('sampled_train_assignment10.csv')

In [62]:
ratings = pd.read_csv('C:/Users/rohit khoja/Desktop/Sem 7/Data ana/project/archive (1)/movies_metadata.csv')

C:\Users\rohit khoja\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
ratings.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
n_users = ratings.user.unique().shape[0]
n_movies = ratings.movie.unique().shape[0]
print(n_users)
print(n_movies)

5178
51


In [7]:
Ratings = ratings.pivot(index = 'user', columns ='movie', values = 'rating').fillna(0)
Ratings.head()

movie,33,40,41,57,70,72,73,74,106,107,...,899,937,945,950,1011,1028,1052,1080,1087,1098
user,,,,,,,,,,,,,,,,,,,,,
1374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
R = Ratings.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

C:\Users\rohit khoja\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [9]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings, k = 50)

In [10]:
sigma = np.diag(sigma)

In [11]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [12]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movie,33,40,41,57,70,72,73,74,106,107,...,899,937,945,950,1011,1028,1052,1080,1087,1098
0,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078429,...,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431,0.078431
1,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039215,...,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216
2,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019607,...,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608
3,0.058824,0.058823,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058813,...,0.058824,0.058824,0.058823,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824
4,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019607,...,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608


In [13]:
def recommend_movies(predictions, userID, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID-1  # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    
    return sorted_user_predictions

In [17]:
predictions = recommend_movies(preds, 1011, ratings, 20)

In [20]:
predictions


movie
483     5.098039
236     0.098481
380     0.098040
526     0.098039
360     0.098039
472     0.098039
57      0.098039
723     0.098039
1011    0.098039
485     0.098039
587     0.098039
564     0.098039
950     0.098039
1087    0.098039
180     0.098039
893     0.098039
937     0.098039
70      0.098039
582     0.098039
41      0.098039
1098    0.098039
33      0.098039
74      0.098039
476     0.098039
1028    0.098039
850     0.098039
1052    0.098039
609     0.098039
1080    0.098039
178     0.098039
135     0.098039
186     0.098039
162     0.098039
72      0.098039
106     0.098039
648     0.098039
73      0.098039
899     0.098039
877     0.098039
607     0.098039
397     0.098039
185     0.098039
402     0.098039
836     0.098039
40      0.098039
766     0.098039
945     0.098039
782     0.098039
831     0.098039
190     0.098039
107     0.098037
Name: 1010, dtype: float64

In [80]:
predictions.to_csv('check.csv',index=False)

In [23]:
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user', 'movie', 'rating']], reader)


# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9837  0.9812  1.0213  1.0098  0.9948  0.9982  0.0154  
Fit time          1.07    1.03    1.08    1.08    1.00    1.05    0.03    
Test time         0.07    0.02    0.03    0.03    0.03    0.04    0.02    


{'test_rmse': array([0.9836638 , 0.98115127, 1.02134613, 1.00984326, 0.99480332]),
 'fit_time': (1.0670723915100098,
  1.033623218536377,
  1.0794591903686523,
  1.075592041015625,
  0.999870777130127),
 'test_time': (0.06964898109436035,
  0.02462148666381836,
  0.031093597412109375,
  0.03128385543823242,
  0.02535414695739746)}

In [26]:
algo.predict(1310, 33)

Prediction(uid=1310, iid=33, r_ui=None, est=4.074650497754109, details={'was_impossible': False})